In [18]:
import numpy as np

#import keras
#import keras.backend as K
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dropout
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
import math
from keras.callbacks import LearningRateScheduler

In [19]:
batch_size = 32 
num_classes = 10
epochs = 10

(x_train, y_train), (x_test, y_test) = cifar10.load_data() 

y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [20]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
train_generator = train_datagen.flow(
        x_train,
        y_train,        
        batch_size=batch_size)

test_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = test_datagen.flow(
        x_test,
        y_test,        
        batch_size=batch_size)

In [26]:
def build_model():
    net_input = Input(shape=x_train.shape[1:])
    
    net = Conv2D(32, (3, 3), padding='same', activation='relu')(net_input)
    net = Dropout(0.2)(net)
    
    net = Conv2D(32,(3,3),padding='same', activation='relu')(net)
    net = MaxPooling2D(pool_size=(2,2))(net)
 
    net = Conv2D(64,(3,3),padding='same',activation='relu')(net)
    net = Dropout(0.2)(net)
 
    net = Conv2D(64,(3,3),padding='same',activation='relu')(net)
    net = MaxPooling2D(pool_size=(2,2))(net)
 
    net = Conv2D(128,(3,3),padding='same',activation='relu')(net)
    net = Dropout(0.2)(net)
 
    net = Conv2D(128,(3,3),padding='same',activation='relu')(net)
    net = MaxPooling2D(pool_size=(2,2))(net)
    
    net = Flatten(name='flatten')(net) 
    net = Dense(1024, activation='relu')(net)
    net = Dense(512, activation='relu')(net)
    net = Dense(256, activation='relu')(net)
    softmax_output = Dense(num_classes, activation='softmax')(net)

    model = Model(net_input, softmax_output)
    
    model.compile(optimizer=SGD(lr=0.0, momentum=0.9, nesterov=True, decay=0.0), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [31]:

def decay_drop(epcoh):
    initial_lrate =0.01
    drop = 0.1
    epoch_drop = 5.0
    lrate = initial_lrate*math.pow(drop, math.floor((1+epochs)/epoch_drop))
    return lrate

In [32]:
lrate = LearningRateScheduler(decay_drop)

In [33]:

lrate = LearningRateScheduler(decay_drop)
callbacks_list = [lrate]

In [34]:
model = build_model()

print(model.summary())

model.fit_generator(
        train_generator,
        steps_per_epoch=len(x_train) // batch_size,
        epochs=epochs,callbacks=callbacks_list,
        validation_data=validation_generator,
        validation_steps=len(x_test) // batch_size)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_16 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_17 (Dropout)         (None, 16, 16, 64)        0         
__________

In [35]:
score = model.evaluate(x_test, y_test, verbose=0)
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 0.2698


In [36]:
score = model.evaluate(x_train, y_train, verbose=0)
print('Train accuracy:', score[1])

Train accuracy: 0.27044
